In [9]:
import os
import torch
from retrievalmodels import RetrievalModel
from modelfinetuning import fine_tune_with_identity
from compute_embedding_celebA import compute_embeddings_from_images

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
PARENT_DIRNAME = os.path.expanduser("~/image-processing-project/")
IMAGE_DIR = os.path.join(PARENT_DIRNAME, "data/img_align_celeba/")
STORAGE_DATA_DIRNAME = os.path.join(PARENT_DIRNAME, "fine_tuning/data_for_fine_tuning")
MODEL_DIR = os.path.join(PARENT_DIRNAME, "fine_tuning/models")

In [12]:
NUM_WORKERS = 4
LEARNING_RATE = 0.001

In [13]:
train_loader = torch.load(os.path.join(STORAGE_DATA_DIRNAME, "train_loader.pth"))
query_loader = torch.load(os.path.join(STORAGE_DATA_DIRNAME, "query_loader.pth"))
gallery_loader = torch.load(os.path.join(STORAGE_DATA_DIRNAME, "gallery_loader.pth"))

/tmp/ipykernel_39250/2467728300.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load(os.path.join(STORAGE_DATA_DIRNAME, "train_loader.pth"))
/tmp/ip

# ResNet50

In [14]:
model = RetrievalModel(backbone="resnet50", embedding_dim=128).to(device)

In [15]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [16]:
fine_tune_with_identity(
    model=model,
    dataloader=train_loader,
    optimizer=optimizer,
    num_epochs=10,
    device=device,
    num_threads=NUM_WORKERS
)

torch.save(model.state_dict(), os.path.join(MODEL_DIR, "resnet50_identity.pth"))

Epoch 1/10: 100%|██████████| 2334/2334 [20:10<00:00,  1.93it/s]


Epoch 1, Triplet Loss: 0.3921


Epoch 2/10: 100%|██████████| 2334/2334 [20:14<00:00,  1.92it/s]


Epoch 2, Triplet Loss: 0.2603


Epoch 3/10: 100%|██████████| 2334/2334 [20:18<00:00,  1.92it/s]


...

Epoch 8, Triplet Loss: 0.1146


Epoch 9/10: 100%|██████████| 2334/2334 [20:07<00:00,  1.93it/s]


Epoch 9, Triplet Loss: 0.1065


Epoch 10/10: 100%|██████████| 2334/2334 [20:13<00:00,  1.92it/s]

Epoch 10, Triplet Loss: 0.1032


In [17]:
model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "resnet50_identity.pth")))

/tmp/ipykernel_39250/4254817467.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "resnet50_identity.pth")))


<All keys matched successfully>

In [18]:
full_embeddings, full_labels = compute_embeddings_from_images(
    model=model,
    device=device
)

torch.save(full_embeddings, os.path.join(STORAGE_DATA_DIRNAME, "full_embeddings_resnet.pth"))
torch.save(full_labels, os.path.join(STORAGE_DATA_DIRNAME, "full_labels_resnet.pth"))

# MobileNetV2

In [4]:
model = RetrievalModel(backbone="mobilenet_v2", embedding_dim=128).to(device)

/home/letruongzzio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/letruongzzio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [24]:
fine_tune_with_identity(
    model=model,
    dataloader=train_loader,
    optimizer=optimizer,
    num_epochs=23,
    device=device,
    num_threads=NUM_WORKERS
)

torch.save(model.state_dict(), os.path.join(MODEL_DIR, "mobilenet_v2_identity.pth"))

Epoch 1/23: 100%|██████████| 2334/2334 [08:24<00:00,  4.62it/s]


Epoch 1, Triplet Loss: 0.4539


Epoch 2/23: 100%|██████████| 2334/2334 [08:27<00:00,  4.60it/s]


Epoch 2, Triplet Loss: 0.3837


Epoch 3/23: 100%|██████████| 2334/2334 [08:28<00:00,  4.59it/s]


...

Epoch 21, Triplet Loss: 0.1940


Epoch 22/23: 100%|██████████| 2334/2334 [08:22<00:00,  4.65it/s]


Epoch 22, Triplet Loss: 0.1870


Epoch 23/23: 100%|██████████| 2334/2334 [08:23<00:00,  4.64it/s]

Epoch 23, Triplet Loss: 0.1794


In [5]:
model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "mobilenet_v2_identity.pth")))

/tmp/ipykernel_50455/1756583994.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(MODEL_DIR, "mobilenet_v2_identity.pth")))


<All keys matched successfully>

In [6]:
full_embeddings, full_labels = compute_embeddings_from_images(
    model=model,
    device=device
)

torch.save(full_embeddings, os.path.join(STORAGE_DATA_DIRNAME, "full_embeddings_mobilenet.pth"))
torch.save(full_labels, os.path.join(STORAGE_DATA_DIRNAME, "full_labels_mobilenet.pth"))